In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/'):
    for filename in filenames:
        print("filename: ", filename)
#         print("middle::", _)
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds

In [ ]:
# import pathlib
# dataset_url = "/kaggle/"
# data_dir = tf.keras.utils.get_file('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9', untar=True)
# data_dir = pathlib.Path(data_dir)

In [ ]:
img_height = 512
img_width = 512
batch_size = 128

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/', 
    validation_split=0.2, subset='training', 
    seed=100,
    image_size=(img_height, img_width)
    #batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/', 
    validation_split=0.2, 
    subset='validation', 
    seed=100,
    image_size=(img_height, img_width)
    #batch_size=batch_size
)
# test_ds = tfds.folder_dataset.ImageFolder('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/')
# train_mask =  tf.keras.utils.image_dataset_from_directory('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_masks/')

In [ ]:
class_names = train_ds.class_names
print(len(class_names))

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=3
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  #batch_size = batch_size
  steps_per_epoch=32
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# code_test=model.predict(test_ds)
# test_ds.class_indices
# train_ds.class_names

class_indices = {k: v for v, k in enumerate(train_ds.class_names)}
    
# print("class_indices::", class_indices)

In [ ]:
#create test ds
import os
test_ds_path = []
for dirname, _, filenames in os.walk('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/'):
    path_dict = {}
    for filename in filenames:
        
#         print(os.path.join(dirname, filename))
        path_dict[filename] = os.path.join(dirname, filename)
    test_ds_path.append(path_dict)
# print("test_ds_path::", test_ds_path)

In [ ]:
def get_top5(oneD_list):
    return np.argpartition(oneD_list,-5)[-5:]

def listOfTop5(twoD_list):
    return [get_top5(row) for row in twoD_list]

key_list = list(class_indices.keys())
val_list = list(class_indices.values())

def replaceWithHotelID(oneRow):
    return [key_list[i] for i in oneRow]

def replace2DwthHotelID(twoDArray):
    return [replaceWithHotelID(row) for row in twoDArray]

#Single Image Prediction Code
def single_image_prediction(submission_filepath):
    
        submission = tf.keras.preprocessing.image.load_img(
            submission_filepath, 
            grayscale=False, 
            color_mode="rgb", 
            target_size=(512, 512), 
            interpolation="nearest"
        )

        submission_arr = tf.keras.preprocessing.image.img_to_array(submission)
#         print("submission_arr::",submission_arr.shape)
        submission_arr = np.array([submission_arr])  # Convert single image to a batch.
        predictions = model.predict(submission_arr)
        print("predictions::", predictions.shape)
        return replace2DwthHotelID(listOfTop5(predictions))

In [ ]:
# single_image_prediction('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/abc.jpg')

prediction_list = []

for path_dict in test_ds_path:
    prediction_dict = {}
    for image, path in path_dict.items():
        prediction_dict["image_id"] = image
#         prediction_dict["hotel_id"] = ''.join(single_image_prediction(path)[0])
        prediction_dict["hotel_id"] = single_image_prediction(path)[0][0]
    prediction_list.append(prediction_dict)
#         print("image::", path)
#     prediction_list['']
print("prediction_list::",  prediction_list)

prediction_df = pd.DataFrame(prediction_list)

In [ ]:
prediction_df.to_csv('submission.csv', index=False)
# prediction_df